# Block Party Freq Solution

## Working with the Ciphertext

Generally, we are more interested in the bytes of our ciphertext than the encoded version of our ciphertext, so when we will read our file in binary mode:

In [1]:
f = open('ciphertext.txt', 'rb') #'r' for read and 'b' for binary mode
ciphertext = f.read()
f.close()

Given we are working with a 16-bit cipher and the same key is applied to each block, we need only find the plaintext of one block. However, without plaintext-ciphertext pairs readily available, we are left guessing pairs. Nevertheless, this kind of guessing game can significantly benefit from a classical cryptanalysis technique known as 'frequency analysis'.

## Using Frequency Analysis

Whenever we read ordinary text, we may notice that 'e', 't', and 'a' are usually the most common letters (in that order). So, if we were dealing with a simple injective encrypting function $E$ where each character is encrypted one-by-one, we would note the most common character in our ciphertext, say 'j', and suppose $E$ maps 'e' to 'j'. Inversely, our decrypting function $D$ would map 'j' to 'e'. We would then suppose $E$ maps 't' to the second most common character in our ciphertext, and 'a' to the third most common. From there, we may be able to pick out some words (think Wheel of Fortune), construct a mapping, and potentially uncover our plaintext. In this case however, we are dealing with two characters at a time given the block size of 16 bits and an 8-bit encoding scheme. So, we need to make use of the most common bigram in the English language, 'th'. Still, we also need to find the most common word (two bytes) in our ciphertext:

In [2]:
cipher_blocks = []
for i in range(0, len(ciphertext), 2):
    cipher_blocks.append(ciphertext[i:i+2])

commonness = 0
for block in set(cipher_blocks):
    if cipher_blocks.count(block) > commonness:
        most_common_block = block
        commonness = cipher_blocks.count(block)

hex(int.from_bytes(most_common_block))

'0x3660'

Now, we may XOR word $3660_{16}$ with $7468_{16}$ ('th' decoded) to suspect a key. Again, this may or may not be the right key as 'th' is not always the most common letter in every piece of text and the encryption function here handles all characters, not just alphabetic ones. Nevertheless, we have the following canidate:

In [3]:
key = int.from_bytes(most_common_block) ^ int.from_bytes(b'th')
print(f'potential key: {hex(key)}')

potential key: 0x4208


## Uncovering our Plaintext

With this suspected key, we may construct plaintext, which may be or not be be the true plaintext. To our fortune, we find:

In [4]:
key_pad = b''.join([key.to_bytes(2)] * len(cipher_blocks))

plaintext = b''
for c, k in zip(ciphertext, key_pad):
    plaintext += (c ^ k).to_bytes()

print(''.join([chr(p) for p in plaintext]))

Good evening,

Below is the information you requested:

flag{5up3r_Fr3q_th3t4}

Nevertheless, I still think there is something off about our current encryption methods. I have a theory about a possible weakness, and I think that I am close to a breakthrough. Still, I wanted to get your thoughts and think up solutions that may soothe any concerns. Hope to hear back soon, thank you.
 


By even cursory inspection, we see rather promising text. Looking closer we find our flag, $\texttt{flag\{5up3r\_Fr3q\_th3t4\}}$.